In [1]:
import random
import os
import subprocess
import shutil
from pathlib import Path

from Bio import AlignIO
from ete3 import Tree

with open("species.list","r") as f:
    lines = f.readlines()

In [3]:
spec = {line.strip() for line in lines}

In [6]:
def parse_fasta(fasta_file):
    """Parse a fasta file and return a dictionary."""
    with open(fasta_file) as f:
        lines = f.readlines()
    fasta_dict = {}
    for line in lines:
        if line.startswith(">"):
            key = line.strip().split()[0][1:]
            fasta_dict[key] = ""
        else:
            fasta_dict[key] += line.strip().upper()
    return fasta_dict

In [7]:
dico = parse_fasta("PF01599.fasta")

In [15]:
### TODO
#Filter the sequences and write them to a file

In [49]:
#Compute MSA and save it as using phylip format
with open("PF01599.aln-phylip") as f:
    ali = AlignIO.read(f, "phylip")

In [58]:
### TODO

#compute 10 bootstrap msa
#boostrap_ali +  ali can be used to concatenate alignments
#ali[:,start:end] can be used to extract columns in an alignment

random.seed(0)
for i in range(10):

    #TODO
    
    with open("bootstrap_msa/PF01599_bootstrap_"+str(i)+".aln-phylip", "w") as output_handle:
        AlignIO.write(boostrap_ali, output_handle, "phylip")

In [87]:
#can be used to run phylip proml if you have correctly installed.

bootstrap_folder = Path("bootstrap_msa")  # folder containing MSA replicates

output_folder = Path("bootstrap_trees")
output_folder.mkdir(exist_ok=True)

batch_commands = """y
y
"""
batch_file = "batch.txt"
with open(batch_file, "w") as f:
    f.write(batch_commands)
    
# Loop over bootstrap files
for msa_file in bootstrap_folder.glob("*.aln-phylip"):
    print(f"Processing {msa_file.name}...")

    phylip_input = "infile"
    # Copy/rename your MSA to the Phylip default input filename
    shutil.copy(msa_file, phylip_input)
    
    # Run proml in batch mode: input from options + MSA
    # The input redirection '<' works differently on Windows cmd, so we'll use subprocess
    with open(batch_file, "r") as f:
        process = subprocess.run(
            ["phylip", "proml"],
            stdin=f,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )
    
    # Rename/move the output tree
    os.remove("outfile")
    outtree = Path("outtree")
    if outtree.exists():
        new_tree_name = output_folder / f"{msa_file.stem}_tree.nwk"
        shutil.move(str(outtree), new_tree_name)
        print(f"Saved tree to {new_tree_name}")
    else:
        print(f"Error: outtree not found for {msa_file.name}")
        print(process.stdout)
        print(process.stderr)

Processing PF01599_bootstrap_0.aln-phylip...
Saved tree to bootstrap_trees/PF01599_bootstrap_0_tree.nwk
Processing PF01599_bootstrap_3.aln-phylip...
Saved tree to bootstrap_trees/PF01599_bootstrap_3_tree.nwk
Processing PF01599_bootstrap_9.aln-phylip...
Saved tree to bootstrap_trees/PF01599_bootstrap_9_tree.nwk
Processing PF01599_bootstrap_1.aln-phylip...
Saved tree to bootstrap_trees/PF01599_bootstrap_1_tree.nwk
Processing PF01599_bootstrap_7.aln-phylip...
Saved tree to bootstrap_trees/PF01599_bootstrap_7_tree.nwk
Processing PF01599_bootstrap_5.aln-phylip...
Saved tree to bootstrap_trees/PF01599_bootstrap_5_tree.nwk
Processing PF01599_bootstrap_2.aln-phylip...
Saved tree to bootstrap_trees/PF01599_bootstrap_2_tree.nwk
Processing PF01599_bootstrap_4.aln-phylip...
Saved tree to bootstrap_trees/PF01599_bootstrap_4_tree.nwk
Processing PF01599_bootstrap_8.aln-phylip...
Saved tree to bootstrap_trees/PF01599_bootstrap_8_tree.nwk
Processing PF01599_bootstrap_6.aln-phylip...
Saved tree to boots

In [89]:
# -------------------------------
# User parameters
# -------------------------------
reference_tree_file = "PF01599_outtree_ref.nwk"
bootstrap_folder = "bootstrap_trees"
output_tree_file = "PF01599_outtree_ref_bootstrapped.nwk"
# -------------------------------

def clade_leafset(node):
    """Return frozenset of leaf names under a node."""
    return frozenset([leaf.name for leaf in node.get_leaves()])

def nontrivial_clades(tree):
    """Return set of all nontrivial clades (size between 2 and N-1)."""
    leaves = tree.get_leaf_names()
    N = len(leaves)
    clades = set()

    for node in tree.traverse():
        s = clade_leafset(node)
        if 1 < len(s) < N:   # ignore trivial splits
            clades.add(s)
    return clades

def load_bootstrap_trees(folder):
    """Load all .nwk or .files from a folder."""
    trees = []
    for fname in os.listdir(folder):
        if fname.endswith(".nwk"):
            t = Tree(os.path.join(folder, fname), format=1)
            trees.append(t)
    return trees


# -------------------------------
# Load reference tree
# -------------------------------
ref_tree = Tree(reference_tree_file, format=1)
ref_clades = nontrivial_clades(ref_tree)

# Map each clade (frozenset) to actual node in reference tree
ref_map = {}
for node in ref_tree.traverse():
    s = clade_leafset(node)
    if s in ref_clades:
        ref_map[s] = node

print(f"Found {len(ref_clades)} informative clades in the reference tree.")

# -------------------------------
# Load bootstrap trees
# -------------------------------
bootstrap_trees = load_bootstrap_trees(bootstrap_folder)
R = len(bootstrap_trees)
print(f"Loaded {R} bootstrap trees.")

if R == 0:
    raise ValueError("No bootstrap trees found! Check folder path.")

# -------------------------------
# Count occurrences of each reference clade
# -------------------------------

### TODO
# count occurrences
counts = {s: 0 for s in ref_clades}
for bt in bootstrap_trees:

    pass

# -------------------------------
# Annotate reference tree
# -------------------------------
for clade_set, node in ref_map.items():
    support_percentage = 100.0 * counts[clade_set] / R
    node.support = support_percentage  # ete3 standard field

# -------------------------------
# Save annotated tree
# -------------------------------
ref_tree.write(outfile=output_tree_file, format=1)
print(f"Annotated tree written to {output_tree_file}")

Found 37 informative clades in the reference tree.
Loaded 10 bootstrap trees.
Annotated tree written to PF01599_outtree_ref_bootstrapped.nwk
